# TF-IDF, BM25, ElasticSearch
# wikipedia로 임베딩을 만들고 valid set에 대해 테스트해보기
## TF-IDF, BM25는 pretrained tokenizer 사용 / Elasticsearch는 내장 nori_tokenizer 사용

In [1]:
from rank_bm25 import BM25Okapi, BM25Plus
from sklearn.feature_extraction.text import TfidfVectorizer
from elasticsearch import Elasticsearch

from transformers import AutoTokenizer
import os
import json
import pickle
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

from datasets import load_from_disk

### Tokenizer 설정

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

### 전처리 함수

In [3]:
def preprocess_retrieval(corpus):
    corpus = corpus.replace("\\n", "")
    corpus = re.sub(f"[\"<>\[\].,?!\(\)\:#\|'\=-]", " ", corpus)
    corpus = ' '.join(corpus.split())
    return corpus

## 데이터 불러오기

In [4]:
# wikipedia_documents.json
with open("../data/wikipedia_documents.json", "r", encoding="utf-8") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))

In [5]:
# train/valid dataset
train_dataset = load_from_disk("../data/train_dataset")
train_context, valid_context = [], []
train_query, valid_query = [], []
for data in tqdm(train_dataset['train']):
    train_context.append(preprocess_retrieval(data['context']))
    train_query.append(preprocess_retrieval(data['question']))
for data in tqdm(train_dataset['validation']):
    valid_context.append(preprocess_retrieval(data['context']))
    valid_query.append(preprocess_retrieval(data['question']))

100%|██████████| 240/240 [00:00<00:00, 5699.59it/s]


In [6]:
print(len(train_context), len(valid_context))
print(train_context[0])

3952 240
미국 상의원 또는 미국 상원 United States Senate 은 양원제인 미국 의회의 상원이다 미국 부통령이 상원의장이 된다 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다 임기는 6년이며 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다 하원이 세금과 경제에 대한 권한 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다 즉 캘리포니아주 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다 그로 인하여 군대의 파병 관료의 임명에 대한 동의 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다 그리고 하원에 대한 견제 역할 하원의 법안을 거부할 권한 등 을 담당한다 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션 공공건강보험기관 의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다 날짜 2017 02 05


In [7]:
train_ids = list(range(len(train_context)))
valid_ids = list(range(len(valid_context)))
len(train_ids), type(valid_ids)

(3952, list)

In [8]:
contexts = [preprocess_retrieval(corpus) for corpus in contexts]

## BM25Okapi 실험

In [52]:
tokenized_wiki = [tokenizer.tokenize(corpus) for corpus in tqdm(contexts)]
bm25 = BM25Okapi(tqdm(tokenized_wiki))

100%|██████████| 56737/56737 [00:07<00:00, 7256.25it/s]


In [53]:
type(bm25)

rank_bm25.BM25Okapi

In [54]:
query = "트랜스포머"
tokenized_query = tokenizer.tokenize(preprocess_retrieval(query))

doc_scores = bm25.get_scores(tokenized_query)
doc_scores

array([0.        , 2.70164591, 3.52023546, ..., 2.56500449, 0.        ,
       0.        ])

In [55]:
bm25.get_top_n(tokenized_query, contexts, n=3)

['트랜스포머 실시영화 세계관에서는 1에서 처음등장하였다 비클모드는 머스탱이다 실사판 영화에서는 몸속의 프렌지를 데리고 다닌다 언뜻 보면 사운드웨이브처럼 숨기고 다닌다 트랜스포머 실사판 3편에서도 잠깐 휠잭를 죽일때나온다 그리고 군대쪽에서 총을 사격하자 그의 눈에 맞는다 그리고 군대쪽에서 빌딩에서 내려오고 군대쪽에서 바리케이드의 발쪽에 이상한 폭파무기를 다리쪽에 심어놓고는 폭파되어 사망한다 그후로 트랜스포머5 최후의 기사 에서도 부활하여 다시 출연한다 하지만 초반에는 메가트론과 좀 나오다가 중간에 시가전을 펼칠때 그림록의 꼬리에 밀려나간 후로부터 나오질 않는다 그 다음에는 스톤헨지에서 잠깐 메가트론 니트로제우스와 함께 포착되고 그 후로부터 등장이 없다 얼라인드 세계관에서 작품인 트랜스포머 워 포 사이버트론에서는 플레이캐릭터로 등장한다',
 '1969년 그렇게 개발된 것이 2254E 컴프레서/리미터이다 2254는 기본적으로 시그널이 게인의 변화를 주는 섹션을 지나서 다시 게인컨트롤 섹션에 영향을 주는 피드백 방식이었다 2254의 특징은 총 4개의 트랜스포머로 가득한 게인스테이지 덩어리라는 것이다 일단 인풋 게인 스테이지에서 첫 번째 트랜스포머가 들어온 신호에 색을 칠한다 그 다음 신호는 게인 리덕션 유닛으로 넘어오는데 니브만의 다이오드 브릿지 diode bridges 방식은 사실 레벨 효율이 매우 떨어지는 방식이었다 게인 리덕션 유닛을 지난 신호는 인풋신호에 비해 거의 40dB 가까이 떨어진채 두 번째 트랜스포머의 색을 묻힌 후 니브의 전매특허 BA283 아웃풋 앰프로 들어가게 된다 BA283에 포함된 총 세 번째 트랜스포머를 지나 원래의 라인신호 레벨을 되찾은 신호는 엔지니어 마음에 따라 리미터 사이드체인으로 들어가거나 말거나 중에 하나를 택한다 리미터까지 들어가면 총 4개의 트랜스포머를 거치게 되는 것이다 그로 인해 2254는 다른 기종에 비해 낮은 노이즈 대 시그널 비율을 가지고 있다 그리고 특유의 색깔도 매우 독특하다 이것이 2254만의 매력이다 게인리덕

In [56]:
def correct_retrieval(method, topk, passages, texts, queries):
    accuracy = [[0, 0] for _ in range(len(topk))]
    for i in tqdm(range(len(texts))):
        tokenized_query = tokenizer.tokenize(preprocess_retrieval(queries[i]))
        predict = method.get_top_n(tokenized_query, passages, n=max(topk))
        for idx, k in enumerate(topk):
            topk_predict = predict[:k]
            if texts[i] in topk_predict:
                accuracy[idx][0] += 1
            else:
                accuracy[idx][1] += 1
    print(f"Total Length : {accuracy[0][0]+accuracy[0][1]}")
    return accuracy

In [57]:
topk_list = [1, 2, 3, 5, 10]
bm25_acc = correct_retrieval(bm25, topk_list, contexts, valid_context, valid_query)

100%|██████████| 240/240 [01:47<00:00,  2.24it/s]

Total Length : 240


In [58]:
for i, topk in enumerate(topk_list):
    print(f"Top-{topk} Acc. : {100*bm25_acc[i][0]/(bm25_acc[i][0]+bm25_acc[i][1]):.2f}")

Top-1 Acc. : 48.33
Top-2 Acc. : 53.33
Top-3 Acc. : 55.83
Top-5 Acc. : 58.75
Top-10 Acc. : 65.00


## BM25Plus 실험

In [36]:
bm25plus = BM25Plus(tqdm(tokenized_wiki))

100%|██████████| 56737/56737 [00:06<00:00, 8444.55it/s]


In [37]:
query = "트랜스포머"
tokenized_query = tokenizer.tokenize(preprocess_retrieval(query))

doc_scores = bm25plus.get_scores(tokenized_query)
doc_scores

array([12.41217118, 16.19212622, 18.9751972 , ..., 12.41217118,
       12.41217118, 12.41217118])

In [38]:
bm25plus.get_top_n(tokenized_query, contexts, n=3)

['트랜스포머 실시영화 세계관에서는 1에서 처음등장하였다 비클모드는 머스탱이다 실사판 영화에서는 몸속의 프렌지를 데리고 다닌다 언뜻 보면 사운드웨이브처럼 숨기고 다닌다 트랜스포머 실사판 3편에서도 잠깐 휠잭를 죽일때나온다 그리고 군대쪽에서 총을 사격하자 그의 눈에 맞는다 그리고 군대쪽에서 빌딩에서 내려오고 군대쪽에서 바리케이드의 발쪽에 이상한 폭파무기를 다리쪽에 심어놓고는 폭파되어 사망한다 그후로 트랜스포머5 최후의 기사 에서도 부활하여 다시 출연한다 하지만 초반에는 메가트론과 좀 나오다가 중간에 시가전을 펼칠때 그림록의 꼬리에 밀려나간 후로부터 나오질 않는다 그 다음에는 스톤헨지에서 잠깐 메가트론 니트로제우스와 함께 포착되고 그 후로부터 등장이 없다 얼라인드 세계관에서 작품인 트랜스포머 워 포 사이버트론에서는 플레이캐릭터로 등장한다',
 '1969년 그렇게 개발된 것이 2254E 컴프레서/리미터이다 2254는 기본적으로 시그널이 게인의 변화를 주는 섹션을 지나서 다시 게인컨트롤 섹션에 영향을 주는 피드백 방식이었다 2254의 특징은 총 4개의 트랜스포머로 가득한 게인스테이지 덩어리라는 것이다 일단 인풋 게인 스테이지에서 첫 번째 트랜스포머가 들어온 신호에 색을 칠한다 그 다음 신호는 게인 리덕션 유닛으로 넘어오는데 니브만의 다이오드 브릿지 diode bridges 방식은 사실 레벨 효율이 매우 떨어지는 방식이었다 게인 리덕션 유닛을 지난 신호는 인풋신호에 비해 거의 40dB 가까이 떨어진채 두 번째 트랜스포머의 색을 묻힌 후 니브의 전매특허 BA283 아웃풋 앰프로 들어가게 된다 BA283에 포함된 총 세 번째 트랜스포머를 지나 원래의 라인신호 레벨을 되찾은 신호는 엔지니어 마음에 따라 리미터 사이드체인으로 들어가거나 말거나 중에 하나를 택한다 리미터까지 들어가면 총 4개의 트랜스포머를 거치게 되는 것이다 그로 인해 2254는 다른 기종에 비해 낮은 노이즈 대 시그널 비율을 가지고 있다 그리고 특유의 색깔도 매우 독특하다 이것이 2254만의 매력이다 게인리덕

In [41]:
bm25plus_acc = correct_retrieval(bm25plus, 1, contexts, valid_context, valid_query)
print(f"{100*bm25plus_acc:.2f}")

100%|██████████| 240/240 [02:13<00:00,  1.80it/s]

Total Length : 240
52.92


## TF-IDF 실험 (max_features 제한 X)

In [43]:
def get_relevant_doc(tfidfv, p_embedding, query, k=1):
    query_vec = tfidfv.transform([query])
    result = query_vec * p_embedding.T
    result = result.toarray()

    sorted_result = np.argsort(result.squeeze())[::-1]
    doc_score = result.squeeze()[sorted_result].tolist()[:k]
    doc_indices = sorted_result.tolist()[:k]
    return doc_score, doc_indices

In [47]:
def retrieval(tfidfv, p_embedding, query_or_dataset, topk):
    doc_scores, doc_indices = get_relevant_doc(tfidfv, p_embedding, query_or_dataset, k=topk)
    answers = []
    for i in range(topk):
        answers.append(contexts[doc_indices[i]])
    return answers

In [46]:
tfidfv = TfidfVectorizer(tokenizer=tokenizer.tokenize, ngram_range=(1, 2))
p_embedding = tfidfv.fit_transform(tqdm(contexts))

100%|██████████| 56737/56737 [02:11<00:00, 430.41it/s]


In [55]:
def tfidf_retrieval(method, embedding, topk, passages, texts, queries):
    right, wrong = 0, 0
    for i in tqdm(range(len(texts))):
        predict = retrieval(tfidfv=tfidfv, p_embedding=embedding, query_or_dataset=queries[i], topk=topk)
        if texts[i] in predict:
            right += 1
        else:
            wrong += 1
    print(f"Total Length : {right+wrong}")
    return right/(right+wrong)

In [51]:
### (주의)시간 오래 걸림 - 약 15분 ###
tfidf_acc = tfidf_retrieval(tfidfv, p_embedding, 1, contexts, valid_context, valid_query)
print(f"{100*tfidf_acc:.2f}")

100%|██████████| 240/240 [14:58<00:00,  3.74s/it]

Total Length : 240
47.92


## Elastic Search 실험 (nori_tokenizer)

In [17]:
!service elasticsearch start

 * Starting Elasticsearch Server
 * Already running.
   ...done.


In [18]:
import pprint  
INDEX_NAME = "toy_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "context" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
      }

  }
}

In [19]:
DOCS = {}
for i in tqdm(range(len(contexts))):
    DOCS[i] = {'context':contexts[i]}

100%|██████████| 56737/56737 [00:00<00:00, 547561.06it/s]


In [20]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [21]:
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '13197ba8cc4f',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'NsjOR1MlSNKPgkiGfKoBIg',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [22]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-22-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-22-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'toy_index'}

In [24]:
import time
for doc_id, doc in DOCS.items():
    es.index(index=INDEX_NAME,  id=doc_id, body=doc)
    if doc_id % 2000 == 0:
        print(f"{100*doc_id/len(DOCS):.2f}% Done!")
    # time.sleep(0.1)

<ipython-input-24-1f3888081606>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME,  id=doc_id, body=doc)


0.00% Done!
3.53% Done!
7.05% Done!
10.58% Done!
14.10% Done!
17.63% Done!
21.15% Done!
24.68% Done!
28.20% Done!
31.73% Done!
35.25% Done!
38.78% Done!
42.30% Done!
45.83% Done!
49.35% Done!
52.88% Done!
56.40% Done!
59.93% Done!
63.45% Done!
66.98% Done!
70.50% Done!
74.03% Done!
77.55% Done!
81.08% Done!
84.60% Done!
88.13% Done!
91.65% Done!
95.18% Done!
98.70% Done!


In [26]:
doc = es.get(index=INDEX_NAME, id=0)
pprint.pprint(doc)

{'_id': '0',
 '_index': 'toy_index',
 '_primary_term': 1,
 '_seq_no': 3761,
 '_source': {'context': '이 문서는 나라 목록이며 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 '
                        '나열하고 있다 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다 첫 번째 부분은 바티칸 시국과 '
                        '팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 '
                        '195개 나라를 나열하고 있다 두 번째 부분은 일부 지역의 주권을 사실상 데 팍토 행사하고 '
                        '있지만 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다 두 목록은 '
                        '모두 가나다 순이다 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며 이 때문에 이러한 '
                        '목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다 이 목록을 구성하고 있는 국가를 '
                        '선정하는 기준에 대한 정보는 포함 기준 단락을 통해 설명하였다 나라에 대한 일반적인 정보는 국가 '
                        '문서에서 설명하고 있다'},
 '_type': '_doc',
 '_version': 2,
 'found': True}


In [27]:
query = "트랜스포머 영화"
res = es.indices.analyze(index=INDEX_NAME,
                                 body={
                                       "analyzer" : "korean",
                                        "text" : query
                                 }
                        )
pprint.pprint(res)

{'tokens': [{'end_offset': 5,
             'position': 0,
             'start_offset': 0,
             'token': '트랜스포머',
             'type': 'word'},
            {'end_offset': 8,
             'position': 0,
             'positionLength': 2,
             'start_offset': 0,
             'token': '트랜스포머 영화',
             'type': 'shingle'},
            {'end_offset': 8,
             'position': 1,
             'start_offset': 6,
             'token': '영화',
             'type': 'word'}]}


In [28]:
query = "트랜스포머 영화"
res = es.search(index=INDEX_NAME, q=query, size=3)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '39799',
                    '_index': 'toy_index',
                    '_score': 21.805086,
                    '_source': {'context': '트랜스포머 실시영화 세계관에서는 1에서 처음등장하였다 '
                                           '비클모드는 머스탱이다 실사판 영화에서는 몸속의 프렌지를 데리고 '
                                           '다닌다 언뜻 보면 사운드웨이브처럼 숨기고 다닌다 트랜스포머 '
                                           '실사판 3편에서도 잠깐 휠잭를 죽일때나온다 그리고 군대쪽에서 '
                                           '총을 사격하자 그의 눈에 맞는다 그리고 군대쪽에서 빌딩에서 '
                                           '내려오고 군대쪽에서 바리케이드의 발쪽에 이상한 폭파무기를 '
                                           '다리쪽에 심어놓고는 폭파되어 사망한다 그후로 트랜스포머5 '
                                           '최후의 기사 에서도 부활하여 다시 출연한다 하지만 초반에는 '
                                           '메가트론과 좀 나오다가 중간에 시가전을 펼칠때 그림록의 꼬리에 '
                                           '밀려나간 후로부터 나오질 않는다 그 다음에는 스톤헨지에서 잠깐 '
                  

In [29]:
res['hits']['hits'][0]['_source']['context']

'트랜스포머 실시영화 세계관에서는 1에서 처음등장하였다 비클모드는 머스탱이다 실사판 영화에서는 몸속의 프렌지를 데리고 다닌다 언뜻 보면 사운드웨이브처럼 숨기고 다닌다 트랜스포머 실사판 3편에서도 잠깐 휠잭를 죽일때나온다 그리고 군대쪽에서 총을 사격하자 그의 눈에 맞는다 그리고 군대쪽에서 빌딩에서 내려오고 군대쪽에서 바리케이드의 발쪽에 이상한 폭파무기를 다리쪽에 심어놓고는 폭파되어 사망한다 그후로 트랜스포머5 최후의 기사 에서도 부활하여 다시 출연한다 하지만 초반에는 메가트론과 좀 나오다가 중간에 시가전을 펼칠때 그림록의 꼬리에 밀려나간 후로부터 나오질 않는다 그 다음에는 스톤헨지에서 잠깐 메가트론 니트로제우스와 함께 포착되고 그 후로부터 등장이 없다 얼라인드 세계관에서 작품인 트랜스포머 워 포 사이버트론에서는 플레이캐릭터로 등장한다'

In [103]:
def right_es(es, texts, queries, topk=1):
    right, wrong = 0, 0
    for i in range(len(queries)):
        try:
            res = es.search(index=INDEX_NAME, q=queries[i], size=topk)
        except:
            mod_q = queries[i].replace("%", " ")
            res = es.search(index=INDEX_NAME, q=mod_q, size=topk)
        answers = []
        for idx in range(topk):
            answers.append(res['hits']['hits'][idx]['_source']['context'])
            
        if texts[i] in answers:
            right += 1
        else:
            wrong += 1
        if i%24 == 0:
            print(f"{100*i/len(texts)}% Done!")
    print(f"Total Length : {right+wrong}")
    return right/(right+wrong)

In [81]:
es_acc = right_es(es, valid_context, valid_query, 100)
print(f"{100*es_acc:.2f}")

0.0% Done!


10.0% Done!
20.0% Done!
30.0% Done!
40.0% Done!
50.0% Done!
60.0% Done!
70.0% Done!
80.0% Done!
90.0% Done!
Total Length : 240
97.92


In [83]:
for q, txt in wrongs:
    print('-'*40, 'Query & Context', '-'*40)
    print(f"Query : {q}")
    print(f"Context : {txt}")

---------------------------------------- Query & Context ----------------------------------------
Query : 설리반이 불만을 표시한 대상은 누구인가
Context : 1778년 초 설리번은 로드아일랜드로 전출되어 그곳의 부대와 민병대를 지휘하게 되었다 그것은 프랑스의 참전이후 난공불락으로 여겨지는 뉴포트를 장악한 영국군을 프랑스 해군과 연계하여 공격하거나 포위하려는 의도였다 그러나 데스탱 장군이 이끄는 함대가 폭우로 손실되어 그 계획은 취소되었다 손실된 배와 하우 제독이 이끄는 영국 함대의 도착으로 데스탱은 보스톤으로 철수하였다 그때 뉴포트의 영국군 요새에 있던 영국군이 출동하였고 설리번도 8월에 일어난 영양가 없었던 로드아일랜드 전투에서 교전을 벌인 후 퇴각했다 난공불락으로 보였던 요새 공략에 실패하고 작전이 붕괴된 방식으로 인해 프랑스와 미합중국 간의 관계가 불편해지게 되었다 설리반은 데스탱 장군에게 편지를 써서 그가 목격했던 것을 ‘프랑스의 명예를 손상시키는’이라는 표현을 써가며 배반적이고 비겁한 짓이라고 항의했다 이 작전 실패로 두 연합국 간의 국제 분쟁에 불이 붙었으며 1년 뒤 사바나 포위전에서 영국 요새에 대한 또 다른 실패를 불러왔다 설리번은 이 실패로 그의 이력에 큰 타격을 입지는 않았으며 그는 조심스런 캐나다 침공을 위한 지휘관으로 오르내리고 있었다
---------------------------------------- Query & Context ----------------------------------------
Query : 아이오와주에 수많은 산업체들이 들어오기 시작한 것은 언제인가
Context : 제2차 세계 대전이 일어나는 동안에 아이오와 주에서 생산된 옥수수와 돼지고기를 포함한 미국 농산품들을 위한 요청이 늘어났다 결과로서 아이오와 주 농부들의 소득이 재빠르게 상승하였다 1945년과 1960년대 후반 사이에 수백개의 새로운 산업들이 아이오와 주로

### 추가 테스트 해볼 내용
1. nori_tokenizer 모든 query 처리 가능하게 preprocess => 크게 할 내용이 없었음 (Mecab 기반)
2. passage 길이 너무 긴 것 삭제 (2500 이상?)
3. 정답이 context 내 어디 쯤에 위치하는지 파악 -> 뒷단이나 앞단 random deletion

## Elasticsearch 긴 문장 제거 후 임베딩 실험

In [43]:
!service elasticsearch start

 * Starting Elasticsearch Server
 * Already running.
   ...done.


In [44]:
INDEX_NAME = "toy_index"

INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "context" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
      }

  }
}

In [47]:
normal_context = []
for context in contexts:
    if len(context) <= 3000:
        normal_context.append(context)
len(normal_context)

56207

In [48]:
DOCS = {}
for i in tqdm(range(len(normal_context))):
    DOCS[i] = {'context':normal_context[i]}

100%|██████████| 56207/56207 [00:00<00:00, 668997.15it/s]


In [49]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [50]:
es.info()

{'name': '13197ba8cc4f',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'NsjOR1MlSNKPgkiGfKoBIg',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [51]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-51-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-51-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'toy_index'}

In [52]:
for doc_id, doc in DOCS.items():
    es.index(index=INDEX_NAME,  id=doc_id, body=doc)
    if doc_id % 2000 == 0:
        print(f"{100*doc_id/len(DOCS):.2f}% Done!")
    # time.sleep(0.1)

<ipython-input-52-9fdec48c77a3>:2: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME,  id=doc_id, body=doc)


0.00% Done!
3.56% Done!
7.12% Done!
10.67% Done!
14.23% Done!
17.79% Done!
21.35% Done!
24.91% Done!
28.47% Done!
32.02% Done!
35.58% Done!
39.14% Done!
42.70% Done!
46.26% Done!
49.82% Done!
53.37% Done!
56.93% Done!
60.49% Done!
64.05% Done!
67.61% Done!
71.17% Done!
74.72% Done!
78.28% Done!
81.84% Done!
85.40% Done!
88.96% Done!
92.52% Done!
96.07% Done!
99.63% Done!


In [53]:
doc = es.get(index=INDEX_NAME, id=0)
pprint.pprint(doc)

{'_id': '0',
 '_index': 'toy_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'context': '이 문서는 나라 목록이며 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 '
                        '나열하고 있다 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다 첫 번째 부분은 바티칸 시국과 '
                        '팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 '
                        '195개 나라를 나열하고 있다 두 번째 부분은 일부 지역의 주권을 사실상 데 팍토 행사하고 '
                        '있지만 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다 두 목록은 '
                        '모두 가나다 순이다 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며 이 때문에 이러한 '
                        '목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다 이 목록을 구성하고 있는 국가를 '
                        '선정하는 기준에 대한 정보는 포함 기준 단락을 통해 설명하였다 나라에 대한 일반적인 정보는 국가 '
                        '문서에서 설명하고 있다'},
 '_type': '_doc',
 '_version': 1,
 'found': True}


In [54]:
query = "트랜스포머 영화"
res = es.indices.analyze(index=INDEX_NAME,
                                 body={
                                       "analyzer" : "korean",
                                        "text" : query
                                 }
                        )
pprint.pprint(res)

{'tokens': [{'end_offset': 5,
             'position': 0,
             'start_offset': 0,
             'token': '트랜스포머',
             'type': 'word'},
            {'end_offset': 8,
             'position': 0,
             'positionLength': 2,
             'start_offset': 0,
             'token': '트랜스포머 영화',
             'type': 'shingle'},
            {'end_offset': 8,
             'position': 1,
             'start_offset': 6,
             'token': '영화',
             'type': 'word'}]}


In [56]:
query = "트랜스포머 영화"
res = es.search(index=INDEX_NAME, q=query, size=3)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '39400',
                    '_index': 'toy_index',
                    '_score': 21.840775,
                    '_source': {'context': '트랜스포머 실시영화 세계관에서는 1에서 처음등장하였다 '
                                           '비클모드는 머스탱이다 실사판 영화에서는 몸속의 프렌지를 데리고 '
                                           '다닌다 언뜻 보면 사운드웨이브처럼 숨기고 다닌다 트랜스포머 '
                                           '실사판 3편에서도 잠깐 휠잭를 죽일때나온다 그리고 군대쪽에서 '
                                           '총을 사격하자 그의 눈에 맞는다 그리고 군대쪽에서 빌딩에서 '
                                           '내려오고 군대쪽에서 바리케이드의 발쪽에 이상한 폭파무기를 '
                                           '다리쪽에 심어놓고는 폭파되어 사망한다 그후로 트랜스포머5 '
                                           '최후의 기사 에서도 부활하여 다시 출연한다 하지만 초반에는 '
                                           '메가트론과 좀 나오다가 중간에 시가전을 펼칠때 그림록의 꼬리에 '
                                           '밀려나간 후로부터 나오질 않는다 그 다음에는 스톤헨지에서 잠깐 '
                  

In [57]:
es_acc = right_es(es, valid_context, valid_query)
print(f"{100*es_acc:.2f}")

0.0% Done!


10.0% Done!
20.0% Done!
30.0% Done!
40.0% Done!
50.0% Done!
60.0% Done!
70.0% Done!
80.0% Done!
90.0% Done!
Total Length : 240
70.00


### 길이가 긴 passage 삭제 -> 효과 없었음

## query에서 키워드만 추출하면 정확도가 올라가는지 실험

In [85]:
val_query = pd.DataFrame({'query':valid_query})
val_query.sample(20)

,query
78,대한민국 민법에서 채권의 권리가 누구에게 있으면 채권이 사라지지 않는다고 명시하는가
179,면역력이 약해져서 아시클로버의 약효가 발휘되지 않을 때 쓰는 것은 무엇인가
50,정민과 이별한 이후 옥림의 매력에 마음을 빼앗겨버린 인물은
19,박지훈은 1라운드에서 몇 순위를 차지했는가
16,몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의 이름은
223,센니치마에 선의 관리권은 어디에 있나
134,돈대에서 무너져내린 포좌부분이 있는 위치는
193,타코벨과 달리 대한민국에서 많은 성과를 달성했던 업체는
170,탄수화물의 최종적 분해가 이루어지는 곳은
188,윤진학이 동아찬영회에 들어가기 전에 속해있었던 곳은


### 그 전에 우선 전처리 다르게 해서 동일 실험 진행해보기

In [143]:
def preprocess(corpus):
    corpus = corpus.replace("\\n", "")
    corpus = re.sub(f"[^- ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Zぁ-ゔァ-ヴー々〆〤一-龥]", " ", corpus)
    corpus = ' '.join(corpus.split())
    return corpus

In [144]:
with open("../data/wikipedia_documents.json", "r", encoding="utf-8") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))

In [145]:
train_dataset = load_from_disk("../data/train_dataset")
new_context, new_query = [], []
for data in tqdm(train_dataset['validation']):
    new_context.append(preprocess(data['context']))
    new_query.append(preprocess(data['question']))

100%|██████████| 240/240 [00:00<00:00, 5211.58it/s]


In [146]:
new_contexts = [preprocess(corpus) for corpus in contexts]

In [147]:
!service elasticsearch start

 * Starting Elasticsearch Server
 * Already running.
   ...done.


In [148]:
INDEX_NAME = "toy_index"

INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],
          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "context" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
      }

  }
}

In [149]:
DOCS = {}
for i in tqdm(range(len(new_contexts))):
    DOCS[i] = {'context':new_contexts[i]}

100%|██████████| 56737/56737 [00:00<00:00, 571957.88it/s]


In [150]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [151]:
es.info()

{'name': '13197ba8cc4f',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'NsjOR1MlSNKPgkiGfKoBIg',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [152]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-152-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-152-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'toy_index'}

In [153]:
for doc_id, doc in DOCS.items():
    es.index(index=INDEX_NAME,  id=doc_id, body=doc)
    if doc_id % 2000 == 0:
        print(f"{100*doc_id/len(DOCS):.2f}% Done!")
    # time.sleep(0.1)

<ipython-input-153-9fdec48c77a3>:2: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME,  id=doc_id, body=doc)


0.00% Done!
3.53% Done!
7.05% Done!
10.58% Done!
14.10% Done!
17.63% Done!
21.15% Done!
24.68% Done!
28.20% Done!
31.73% Done!
35.25% Done!
38.78% Done!
42.30% Done!
45.83% Done!
49.35% Done!
52.88% Done!
56.40% Done!
59.93% Done!
63.45% Done!
66.98% Done!
70.50% Done!
74.03% Done!
77.55% Done!
81.08% Done!
84.60% Done!
88.13% Done!
91.65% Done!
95.18% Done!
98.70% Done!


In [154]:
doc = es.get(index=INDEX_NAME, id=0)
pprint.pprint(doc)

{'_id': '0',
 '_index': 'toy_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'context': '이 문서는 나라 목록이며 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 '
                        '나열하고 있다 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다 첫 번째 부분은 바티칸 시국과 '
                        '팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 '
                        '195개 나라를 나열하고 있다 두 번째 부분은 일부 지역의 주권을 사실상 데 팍토 행사하고 '
                        '있지만 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다 두 목록은 '
                        '모두 가나다 순이다 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며 이 때문에 이러한 '
                        '목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다 이 목록을 구성하고 있는 국가를 '
                        '선정하는 기준에 대한 정보는 포함 기준 단락을 통해 설명하였다 나라에 대한 일반적인 정보는 국가 '
                        '문서에서 설명하고 있다'},
 '_type': '_doc',
 '_version': 1,
 'found': True}


In [155]:
query = "트랜스포머 영화"
res = es.indices.analyze(index=INDEX_NAME,
                                 body={
                                       "analyzer" : "korean",
                                        "text" : query
                                 }
                        )
pprint.pprint(res)

{'tokens': [{'end_offset': 5,
             'position': 0,
             'start_offset': 0,
             'token': '트랜스포머',
             'type': 'word'},
            {'end_offset': 8,
             'position': 1,
             'start_offset': 6,
             'token': '영화',
             'type': 'word'}]}


In [156]:
query = "트랜스포머 영화"
res = es.search(index=INDEX_NAME, q=query, size=3)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '39799',
                    '_index': 'toy_index',
                    '_score': 20.270441,
                    '_source': {'context': '트랜스포머 실시영화 세계관에서는 1에서 처음등장하였다 '
                                           '비클모드는 머스탱이다 실사판 영화에서는 몸속의 프렌지를 데리고 '
                                           '다닌다 언뜻 보면 사운드웨이브처럼 숨기고 다닌다 트랜스포머 '
                                           '실사판 3편에서도 잠깐 휠잭를 죽일때나온다 그리고 군대쪽에서 '
                                           '총을 사격하자 그의 눈에 맞는다 그리고 군대쪽에서 빌딩에서 '
                                           '내려오고 군대쪽에서 바리케이드의 발쪽에 이상한 폭파무기를 '
                                           '다리쪽에 심어놓고는 폭파되어 사망한다 그후로 트랜스포머5 '
                                           '최후의 기사 에서도 부활하여 다시 출연한다 하지만 초반에는 '
                                           '메가트론과 좀 나오다가 중간에 시가전을 펼칠때 그림록의 꼬리에 '
                                           '밀려나간 후로부터 나오질 않는다 그 다음에는 스톤헨지에서 잠깐 '
                  

In [158]:
es_acc = right_es(es, new_context, new_query, topk=2)
print(f"{100*es_acc:.2f}")

0.0% Done!
10.0% Done!
20.0% Done!
30.0% Done!
40.0% Done!
50.0% Done!
60.0% Done!
70.0% Done!
80.0% Done!
90.0% Done!
Total Length : 240
76.67


### 특수문자 제거 성능 >> 특수문자 미제거 성능

In [39]:
topk = [1, 2, 3, 5, 10]
accuracy = [[0, 0] for _ in range(len(topk))]
accuracy

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]